# खर्च दावा विश्लेषण

हे नोटबुक दाखवते की प्लगिन्स वापरून एजंट्स कसे तयार करायचे जे स्थानिक पावत्यांच्या प्रतिमांमधून प्रवास खर्च प्रक्रिया करतात, खर्च दावा ईमेल तयार करतात, आणि पाई चार्ट वापरून खर्च डेटा व्हिज्युअलाइज करतात. एजंट्स कार्याच्या संदर्भानुसार डायनॅमिकली फंक्शन्स निवडतात.

पायऱ्या:
1. OCR एजंट स्थानिक पावती प्रतिमा प्रक्रिया करतो आणि प्रवास खर्च डेटा काढतो.
2. ईमेल एजंट खर्च दावा ईमेल तयार करतो.

### प्रवास खर्चाच्या परिस्थितीचे उदाहरण:
कल्पना करा की तुम्ही एका व्यवसायिक बैठकीसाठी दुसऱ्या शहरात प्रवास करणारे कर्मचारी आहात. तुमच्या कंपनीची धोरणे सर्व वाजवी प्रवास-संबंधित खर्चांची परतफेड करण्याची आहेत. प्रवास खर्चाचा संभाव्य तपशील असा आहे:
- वाहतूक:
तुमच्या घराच्या शहरातून गंतव्य शहरापर्यंतच्या राउंड ट्रिपसाठी विमान प्रवास.
एअरपोर्टपर्यंत आणि परत जाण्यासाठी टॅक्सी किंवा राईड-हेलिंग सेवा.
गंतव्य शहरातील स्थानिक वाहतूक (जसे की सार्वजनिक वाहतूक, भाड्याने घेतलेली वाहने, किंवा टॅक्सी).

- निवास:
बैठकीच्या ठिकाणाजवळील मध्यम श्रेणीच्या व्यवसायिक हॉटेलमध्ये तीन रात्रींची राहण्याची व्यवस्था.

- भोजन:
कंपनीच्या पर डाएम धोरणानुसार नाश्ता, दुपारचे जेवण, आणि रात्रीच्या जेवणासाठी दररोजचा भत्ता.

- इतर खर्च:
एअरपोर्टवरील पार्किंग शुल्क.
हॉटेलमधील इंटरनेट प्रवेश शुल्क.
टीप्स किंवा लहान सेवा शुल्क.

- दस्तऐवजीकरण:
तुम्ही सर्व पावत्या (उदा. फ्लाइट्स, टॅक्सी, हॉटेल, भोजन इत्यादी) आणि परतफेडीसाठी पूर्ण केलेला खर्च अहवाल सादर करता.


## आवश्यक लायब्ररी आयात करा

नोटबुकसाठी आवश्यक लायब्ररी आणि मॉड्यूल्स आयात करा.


In [1]:
import os
from dotenv import load_dotenv
from azure.ai.inference import ChatCompletionsClient
from azure.core.credentials import AzureKeyCredential
from semantic_kernel.kernel import Kernel
from semantic_kernel.agents import AgentGroupChat
from openai import AsyncOpenAI
from semantic_kernel.agents import ChatCompletionAgent, AgentGroupChat


from semantic_kernel.contents.utils.author_role import AuthorRole
from semantic_kernel.agents.strategies import SequentialSelectionStrategy, DefaultTerminationStrategy
from semantic_kernel.contents.chat_message_content import ChatMessageContent
from semantic_kernel.contents import ImageContent, TextContent
from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion, OpenAIChatPromptExecutionSettings

from semantic_kernel.functions import kernel_function, KernelArguments
from pydantic import BaseModel, Field
from typing import List
from azure.ai.inference.models import SystemMessage, UserMessage, TextContentItem, ImageContentItem, ImageUrl, ImageDetailLevel

load_dotenv()

True

In [2]:
def _create_kernel_with_chat_completion(service_id: str) -> Kernel:
    kernel = Kernel()
   
    client = AsyncOpenAI(
    api_key=os.environ["GITHUB_TOKEN"], base_url="https://models.inference.ai.azure.com/")
    kernel.add_service(
        OpenAIChatCompletion(
            ai_model_id="gpt-4o-mini",
            async_client=client,
            service_id="open_ai"
        )
    )

    kernel.add_service(
        OpenAIChatCompletion(
            ai_model_id="gpt-4o",
            async_client=client,
            service_id="gpt-4o"
        )
    )

    return kernel

## खर्च मॉडेल्स परिभाषित करा

 वैयक्तिक खर्चांसाठी Pydantic मॉडेल तयार करा आणि वापरकर्त्याच्या क्वेरीला संरचित खर्च डेटा मध्ये रूपांतरित करण्यासाठी एक ExpenseFormatter वर्ग तयार करा.

 प्रत्येक खर्च खालील स्वरूपात दर्शविला जाईल:
 `{'date': '07-Mar-2025', 'description': 'गंतव्यस्थानासाठी विमान प्रवास', 'amount': 675.99, 'category': 'Transportation'}`


In [3]:
class Expense(BaseModel):
    date: str = Field(..., description="Date of expense in dd-MMM-yyyy format")
    description: str = Field(..., description="Expense description")
    amount: float = Field(..., description="Expense amount")
    category: str = Field(..., description="Expense category (e.g., Transportation, Meals, Accommodation, Miscellaneous)")

class ExpenseFormatter(BaseModel):
    raw_query: str = Field(..., description="Raw query input containing expense details")
    
    def parse_expenses(self) -> List[Expense]:
        """
        Parses the raw query into a list of Expense objects.
        Expected format: "date|description|amount|category" separated by semicolons.
        """
        expense_list = []
        for expense_str in self.raw_query.split(";"):
            if expense_str.strip():
                parts = expense_str.strip().split("|")
                if len(parts) == 4:
                    date, description, amount, category = parts
                    try:
                        expense = Expense(
                            date=date.strip(),
                            description=description.strip(),
                            amount=float(amount.strip()),
                            category=category.strip()
                        )
                        expense_list.append(expense)
                    except ValueError as e:
                        print(f"[LOG] Parse Error: Invalid data in '{expense_str}': {e}")
        return expense_list

## एजंट्स परिभाषित करणे - ईमेल तयार करणे

खर्चाचा दावा सादर करण्यासाठी ईमेल तयार करण्यासाठी एक एजंट वर्ग तयार करा.  
- हा एजंट `kernel_function` डेकोरेटरचा वापर करून खर्चाचा दावा सादर करण्यासाठी ईमेल तयार करणारे फंक्शन परिभाषित करतो.  
- तो खर्चांची एकूण रक्कम मोजतो आणि तपशील ईमेलच्या मजकुरात स्वरूपित करतो.  


In [4]:
class ExpenseEmailAgent:

    @kernel_function(description="Generate an email to submit an expense claim to the Finance Team")
    async def generate_expense_email(expenses):
        total_amount = sum(expense['amount'] for expense in expenses)
        email_body = "Dear Finance Team,\n\n"
        email_body += "Please find below the details of my expense claim:\n\n"
        for expense in expenses:
            email_body += f"- {expense['description']}: ${expense['amount']}\n"
        email_body += f"\nTotal Amount: ${total_amount}\n\n"
        email_body += "Receipts for all expenses are attached for your reference.\n\n"
        email_body += "Thank you,\n[Your Name]"
        return email_body

# पावती प्रतिमांमधून प्रवास खर्च काढण्यासाठी एजंट

पावती प्रतिमांमधून प्रवास खर्च काढण्यासाठी एक एजंट वर्ग तयार करा.
- हा एजंट `kernel_function` डेकोरेटर वापरतो ज्याद्वारे पावती प्रतिमांमधून प्रवास खर्च काढण्याचे कार्य परिभाषित केले जाते.
- OCR (ऑप्टिकल कॅरेक्टर रिकग्निशन) वापरून पावती प्रतिमा मजकूरात रूपांतरित करा आणि तारीख, वर्णन, रक्कम, आणि श्रेणी यासारखी संबंधित माहिती काढा.


In [5]:
class OCRAgentPlugin:
    def __init__(self):
        self.client = ChatCompletionsClient(
            endpoint="https://models.inference.ai.azure.com/",
            credential=AzureKeyCredential(os.environ.get("GITHUB_TOKEN")),
        )
        self.model_name = "gpt-4o"

    @kernel_function(description="Extract structured travel expense data from receipt.jpg using gpt-4o-model")
    def extract_text(self, image_path: str = "receipt.jpg") -> str:
        try:
            image_url_str = str(ImageUrl.load(image_file=image_path, image_format="jpg", detail=ImageDetailLevel.HIGH))

            prompt = (
                "You are an expert OCR assistant specialized in extracting structured data from receipt images. "
                "Analyze the provided receipt image and extract travel-related expense details in the format: "
                "'date|description|amount|category' separated by semicolons. "
                "Follow these rules: "
                "- Date: Convert dates (e.g., '4/4/22') to 'dd-MMM-yyyy' (e.g., '04-Apr-2022'). "
                "- Description: Extract item names (e.g., 'Carlson's Drylawn', 'Peigs transaction Probiotics'). "
                "- Amount: Use numeric values (e.g., '4.50' from '$4.50' or '4.50 dollars'). "
                "- Category: Infer from context (e.g., 'Meals' for food, 'Transportation' for travel, 'Accommodation' for lodging, 'Miscellaneous' otherwise). "
                "Ignore totals, subtotals, or service charges unless they are itemized expenses. "
                "If no expenses are found, return 'No expenses detected'. "
                "Return only the structured data, no additional text."
            )
            response = self.client.complete(
                messages=[
                    SystemMessage(content=prompt),
                    UserMessage(content=[
                        TextContentItem(text="Extract travel expenses from this receipt image."),
                        ImageContentItem(image_url=ImageUrl(url=image_url_str))
                    ])
                ],
                model=self.model_name,
                temperature=0.1,
                max_tokens=2048
            )
            extracted_text = response.choices[0].message.content
            return extracted_text
        except Exception as e:
            error_msg = f"[LOG] OCR Plugin: Error processing image: {str(e)}"
            print(error_msg)
            return error_msg

## खर्च प्रक्रिया

खर्च प्रक्रिया करण्यासाठी आवश्यक एजंट तयार करून आणि नोंदणी करून त्यांना सक्रिय करण्यासाठी एक असिंक्रोनस फंक्शन परिभाषित करा.
- हे फंक्शन खर्च प्रक्रिया करते, ज्यामध्ये पर्यावरणीय व्हेरिएबल्स लोड करणे, आवश्यक एजंट तयार करणे आणि त्यांना प्लगइन म्हणून नोंदणी करणे समाविष्ट आहे.
- हे दोन एजंटसह एक गट चॅट तयार करते आणि खर्च डेटा आधारित ईमेल आणि पाई चार्ट तयार करण्यासाठी एक प्रॉम्प्ट संदेश पाठवते.
- चॅट सक्रियतेदरम्यान उद्भवणाऱ्या कोणत्याही त्रुटींचे व्यवस्थापन करते आणि एजंट्सची योग्य साफसफाई सुनिश्चित करते.


In [6]:
async def process_expenses():
    load_dotenv()
    settings_slm = OpenAIChatPromptExecutionSettings(service_id="gpt-4o")
    settings_llm = OpenAIChatPromptExecutionSettings(service_id="open_ai")  # Fixed typo in service_id
    
    ocr_agent = ChatCompletionAgent(
        kernel=_create_kernel_with_chat_completion("ocrAgent"),
        name="ocr_agent",
        instructions="Extract travel expense data from the receipt image in the prompt using the 'extract_text' function from the 'ocrAgent' plugin. Return the data in the format 'date|description|amount|category' separated by semicolons.",
        arguments=KernelArguments(settings=settings_slm)
    )
    
       
    email_agent = ChatCompletionAgent(
            kernel=_create_kernel_with_chat_completion("expenseEmailAgent"),
            name="email_agent",
            instructions="Take the travel expense data from the previous agent and generate a professional expense claim email using the 'generate_expense_email' function from the 'expenseEmailAgent' plugin, then pass the data forward.",
            arguments=KernelArguments(
                settings=settings_llm)
        )


    kernel = Kernel()

    # Use fixed path to receipt.jpg in the same folder
    image_path = "./receipt.jpg"
    
    # Create a structured message with text and image content for OCR processing
    image_url_str = f"file://{image_path}"
    
    # Using the correct format for multi-modal content
    user_message = ChatMessageContent(
        role=AuthorRole.USER,
        items=[
            TextContent(text="""
            Please extract the raw text from this receipt image, focusing on travel expenses like dates, descriptions, amounts, and categories (e.g., Transportation, Accommodation, Meals, Miscellaneous).
            Then generate a professional expense claim email.
                        """),
            ImageContent.from_image_file(path=image_path)
        ]
    )

    # Register plugins with the kernel
    kernel.add_plugin(OCRAgentPlugin(), plugin_name="ocrAgent")
    kernel.add_plugin(ExpenseEmailAgent(), plugin_name="expenseEmailAgent")

    # Create group chat
    chat = AgentGroupChat(
        agents=[ocr_agent, email_agent],
        selection_strategy=SequentialSelectionStrategy(initial_agent=ocr_agent),
        termination_strategy=DefaultTerminationStrategy(maximum_iterations=1)
    )

    # Add user message with prompt
    await chat.add_chat_message(user_message)
    print(f"# User message added to chat with receipt image")

    async for content in chat.invoke():
        print(f"# Agent - {content.name or '*'}: '{content.content}'")


## मुख्य फंक्शन

मुख्य फंक्शन परिभाषित करा ज्यामुळे कन्सोल साफ होईल आणि `process_expenses` फंक्शन असिंक्रोनस पद्धतीने चालवले जाईल.


In [9]:
async def main():
    # Clear the console
    os.system('cls' if os.name=='nt' else 'clear')

    # Run the async agent code
    await process_expenses()

await main()

# User message added to chat with receipt image
# Agent - ocr_agent: 'The receipt primarily seems to capture costs for meals and beverages. Below is the extracted travel expense data:

**Travel Expense Data:**  
`2 May '22|Meals at restaurant|75.15|Meals`

---

**Professional Expense Claim Email Draft:**  

**Subject:** Expense Claim for Meals – 2 May 2022  

Dear [Recipient's Name],  

I am submitting an expense claim for a meal incurred during a business-related trip. Below are the details:  

- **Date:** 2 May 2022  
- **Expense Description:** Meals at a restaurant  
- **Amount:** $75.15  
- **Category:** Meals  

Please find the attached receipt for your reference. Kindly process the reimbursement at your earliest convenience. Let me know if you require additional information.  

Thank you for your assistance.  

Best regards,  
[Your Name]  
[Your Contact Information]  

Let me know if you need further revisions or additional details!'



---

**अस्वीकरण**:  
हा दस्तऐवज AI भाषांतर सेवा [Co-op Translator](https://github.com/Azure/co-op-translator) चा वापर करून भाषांतरित करण्यात आला आहे. आम्ही अचूकतेसाठी प्रयत्नशील असलो तरी, कृपया लक्षात घ्या की स्वयंचलित भाषांतरांमध्ये त्रुटी किंवा अचूकतेचा अभाव असू शकतो. मूळ भाषेतील दस्तऐवज हा अधिकृत स्रोत मानला जावा. महत्त्वाच्या माहितीसाठी, व्यावसायिक मानवी भाषांतराची शिफारस केली जाते. या भाषांतराचा वापर केल्यामुळे उद्भवलेल्या कोणत्याही गैरसमज किंवा चुकीच्या अर्थासाठी आम्ही जबाबदार राहणार नाही.
